In [1]:
import re
import sys
import pydocx
import codecs
import pymysql
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
def p_discriminate(p):
    if str(p).find("img") != -1:
        return (0, "pic")
    else:
        return (1, "book")

In [19]:
class tiban():
    '''
    a tiban class represents a tiban's basic structural information including imgs and the number of tiban
    '''
    def __init__(self, input_file, output_file):
        '''
        init the class
        para: input_file: the file need to be rearrange 
        pare: output_file: the file after rearrange
        '''
        self.tiban_html = pydocx.PyDocX.to_html(input_file)
        self.tiban_soup = BeautifulSoup(self.tiban_html)
        self.input = input_file
        self.output = output_file
        self.errors = []
        self.character = {}
        self.ch_dict = {}
        self.p_list = self.tiban_soup.select('p')
    
    def parse(self):
        '''
        parse the file (html form)
        para: self
        '''
        p_list = self.p_list
        flag = 0
        # if the pre_stage = 1 means that the situation is booknum, 0 means the pictures
        pre_stage = 0
        pre_tag = 0
        this_booknum = ""
        this_character = []
        pre_num = ""
        for p in tqdm(p_list):
            if flag%2 == 0:
                this_booknum = ""
            else:
                this_character = []
            if flag%2 == 0:
                stage = p_discriminate(p)[0]
                if stage == 1:
                    if stage != pre_stage:
                        this_booknum = p.text
                        pre_stage = stage
                    else:
                        self.errors.append(("error 1 多重简号或者换行符有误", pre_num, p.text, "flag="+str(flag)))
                        return flag
                else:
                    self.errors.append(("error 2 多重简号或者字体截图中换行符有误", pre_num, p.text, "flag="+str(flag)))
                    return flag
            else:
                stage = p_discriminate(p)[0]
                if stage == 0:
                    children = p.children
                    tag = 1 # 0: character, 1:image
                    cflag = 0
                    for child in children:
                        # if str(child)
                        if cflag%2 ==0:
                            if str(child).find("img") == -1:
                                tag = 0
                                pre_tag = tag
                                tmp_pair_character = str(child).strip()
                            else:
                                self.errors.append(("error 3 出现连续文字结点", this_booknum, p.text, "flag="+str(flag)))
                                return flag
                        else:
                            if str(child).find("img") != -1:
                                tag = 1
                                if tag != pre_tag:
                                    tmp_pair_image = str(child)
                                else:
                                    self.errors.append(("error 4 出现连续图片", this_booknum, p.text, "flag="+str(flag)))
                                    return flag
                            else:
                                self.errors.append(("error 5 出现连续文字结点", this_booknum, str(child), "flag="+str(flag)))
                                return flag
                
                            if len(tmp_pair_character)!=0 and len(tmp_pair_image)!=0:
                                this_character.append((tmp_pair_character, tmp_pair_image))
                        cflag+=1
                    pre_stage = stage
                else:
                    self.errors.append(("error 6 多重简号或者换行符有误", this_booknum, p.text, "flag=",flag))
                    # return flag
                    flag-=1
            
            if len(this_booknum)!=0 and len(this_character)!=0 and flag%2==1:
                # print(flag, len(this_booknum),len(this_character))
                self.character[this_booknum] = this_character
            pre_num = this_booknum
            flag += 1
    
    def array2dict(self):
        array = self.character
        self.ch_dict = {}
        for k in array.keys():
            for item in array[k]:
                if item[0] not in self.ch_dict.keys():
                    self.ch_dict[item[0]] = []
                    self.ch_dict[item[0]].append((item[1], k))
                else:
                    self.ch_dict[item[0]].append((item[1], k))
                    
    def get_error(self):
        return self.errors
    
    def get_output_html(self):
        f = open(self.output, "w", encoding="utf8")
        for k in self.ch_dict.keys():
            print("<p>"+k+"</p>", file=f)
            print("<p>", end="", file=f)
            for item in self.ch_dict[k]:
                print(item[0], end="", file=f)
                print(item[1], end=",", file=f)
            print("</p>", file=f)
        f.close()

In [26]:
tb = tiban('73EJT.docx', '73EJT.html')
tb.parse()
tb.array2dict()
tb.get_output_html()

100%|██████████| 4/4 [00:00<00:00, 34.40it/s]


In [27]:
tb.errors

[]

In [25]:
print(len(tb.p_list))

2


In [126]:
if __name__=='__main__':
    tb = tiban('../data/test0.docx', 'test0_out.docx')